### Import bibliotek

In [24]:
import requests
from bs4 import BeautifulSoup
import json
import os

NARZĘDZIA

In [25]:
def extract(ancestor, selector, attribute=None, many=False):
    if selector:
      if many:
         if attribute:
               return [item[attribute].strip() for item in ancestor.select(selector)]

         return [item.text.strip() for item in ancestor.select(selector)]
      if attribute:
          try: 
              return ancestor.select_one(selector)[attribute].strip()
          except TypeError:
             return None
      try: 
          return ancestor.select_one(selector).text.strip()
      except AttributeError:
         return None
    if attribute:
       try:
          return ancestor[attribute]
       except TypeError:
          return None
    return ancestor.text.strip()
   
    
      

In [26]:
review_schema = {
    "review_id": (None, 'data-entry-id'),
    "author": ("span.user-post__author-name",),
    "recommendation": ("span.user-post__author-recomendation > em",),
    "stars": ("span.user-post__score-count",),
    "content": ("div.user-post__text",),
    "pros": ("div.review-feature__item--positive", None, True),
    "cons": ("div.review-feature__item--negative", None, True),
    "likes": ("button.vote-yes > span",),
    "dislikes": ("button.vote-no > span",),
    "publish_date": ("span.user-post__published > time:nth-child(1)",'datetime'),
    "purchase_date": ("span.user-post__published > time:nth-child(2)",'datetime'),
}

Headers json

In [27]:
with open("./headers.json", "r") as f:
    headers = json.load(f)

pętla - kroki 1-4 z readme

In [28]:
product_id = "178346457"
next_page = f"https://www.ceneo.pl/{product_id}#tab=reviews"
all_reviews = []
while next_page:
    response = requests.get(next_page, headers=headers)
    print(next_page)
    if response.status_code == 200:
        page_dom = BeautifulSoup(response.text, "html.parser")
        reviews = page_dom.select("div.js_product-review:not(.user-post--highlight)")
        print(len(reviews))
        for review in reviews:
           single_review = {
              key: extract(review, *value)
              for key, value in review_schema.items()
           }
           all_reviews.append(single_review)
            
        try:
           next_page = "https://www.ceneo.pl"+extract(page_dom, "a.pagination__next","href")
        except TypeError:
          next_page = None
        print(all_reviews)


        

   






https://www.ceneo.pl/178346457#tab=reviews
4
[{'review_id': '19493916', 'author': 'b...9', 'recommendation': 'Polecam', 'stars': '5/5', 'content': 'Dobra, głęboka analiza, realistyczna. Zawsze takie głosy są odosobnione.', 'pros': ['akcja', 'fabuła', 'jakość wydania'], 'cons': [], 'likes': '0', 'dislikes': '0', 'publish_date': '2025-03-06 08:27:28', 'purchase_date': '2025-02-27 11:00:47'}, {'review_id': '19476757', 'author': 'y...2', 'recommendation': 'Polecam', 'stars': '5/5', 'content': 'Obowiązkowa pozycja dla tych, co chcą zrozumieć świat.', 'pros': ['akcja', 'fabuła', 'jakość wydania'], 'cons': [], 'likes': '1', 'dislikes': '0', 'publish_date': '2025-02-26 11:19:04', 'purchase_date': '2025-02-14 11:53:54'}, {'review_id': '19504384', 'author': 'Joanna', 'recommendation': 'Polecam', 'stars': '5/5', 'content': 'książka bardzo ciekawa, prezent dla taty ;-)', 'pros': [], 'cons': [], 'likes': '1', 'dislikes': '0', 'publish_date': '2025-03-11 21:04:50', 'purchase_date': '2025-03-04 23:40

Zapisanie wszystkich opinii w bazie danych

In [29]:
if not os.path.exists("./opinions"):
    os.mkdir("./opinions")

In [30]:
with open(f"./opinions/{product_id}.json", "w", encoding="UTF-8") as f:
    json.dump(all_reviews,f,indent=4,ensure_ascii=False)